In [133]:
#Import appropriate libraries
import numpy as np
import pandas as pd
import os
from numpy import nan as NA
import datetime as datetime
from models import db, Student, Floor, Collections, ShelfReading

In [134]:
#New file path folder
excel_file_path = 'file.xlsx'

folder_name = os.path.splitext(os.path.basename(excel_file_path))[0]
parent_directory = "floor_3_collections" 

folder_path = os.path.join(parent_directory, folder_name)

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

In [135]:
#Read excel file of shelfreading data
all_sheets = pd.read_excel('floor_3_SR.xlsx', sheet_name=None)

In [136]:
#Save each sheet as a separate CSV file
for sheet_name, df in all_sheets.items():
    file_path = os.path.join(folder_path, f"{sheet_name}.csv")
    df.to_csv(file_path, index=False)

print(f"All sheets saved as CSV files in the folder: {folder_path}")

All sheets saved as CSV files in the folder: floor_3_collections\file


In [137]:
student_listing = pd.read_csv('Active CMR Student List - Sheet1.csv', header=None)
student_listing[['Last Name', 'First Name']] = student_listing[0].str.split(',', expand=True)
student_listing['First Name'] = student_listing['First Name'].str.strip()
student_listing['Last Name'] = student_listing['Last Name'].str.strip()
new_student_listing = student_listing.drop(columns=[0])

In [138]:
folder_name = os.path.splitext(os.path.basename(excel_file_path))[0] + "_edited"
parent_directory = "new_floor_3" 
folder_path = os.path.join(parent_directory, folder_name)

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

In [139]:
def get_student_id(first_name, last_name):
    student = Student.query.filter_by(student_fname=first_name, student_lname=last_name).first()
    if student is None:
        print(f"Warning: Student with name {first_name} {last_name} not found!")
        return None
    print(f"Found student {first_name} {last_name} with ID {student.student_id}")  # Add logging here
    return student.student_id

In [140]:
def edit_sheet(df, new_student_listing):
    default_datetime = pd.to_datetime('1900-01-01 00:00:00')

    # Select relevant columns from the input dataframe
    temp_df = df.iloc[:, [0, 1, 2, 3, 5, 6, 7]]
    temp_df.columns = ['Name', 'date', 'start_time', 'end_time', 'shelves_completed', 'start_call', 'end_call']

    # Remove rows with all NaN values and reset index
    cleaned_temp_df = temp_df.dropna(how='all').reset_index(drop=True)

    # Convert 'date' column to datetime64[ns] type
    cleaned_temp_df['date'] = pd.to_datetime(cleaned_temp_df['date'], errors='coerce')

    # Convert 'start_time' and 'end_time' to time format
    cleaned_temp_df['start_time'] = pd.to_datetime(cleaned_temp_df['start_time'], format='%H:%M:%S', errors='coerce').dt.time
    cleaned_temp_df['end_time'] = pd.to_datetime(cleaned_temp_df['end_time'], format='%H:%M:%S', errors='coerce').dt.time

    # Combine 'date' with 'start_time' and 'end_time' to form full datetime values
    cleaned_temp_df['Start DateTime'] = cleaned_temp_df.apply(
        lambda row: datetime.combine(row['date'].date(), row['start_time']) if pd.notnull(row['date']) and pd.notnull(row['start_time']) else default_datetime,
        axis=1
    )
    cleaned_temp_df['End DateTime'] = cleaned_temp_df.apply(
        lambda row: datetime.combine(row['date'].date(), row['end_time']) if pd.notnull(row['date']) and pd.notnull(row['end_time']) else default_datetime,
        axis=1
    )

    # Calculate 'Duration' as the difference between 'End DateTime' and 'Start DateTime'
    cleaned_temp_df['Duration'] = cleaned_temp_df.apply(
        lambda row: row['End DateTime'] - row['Start DateTime'] if pd.notnull(row['End DateTime']) and pd.notnull(row['Start DateTime']) else pd.NaT,
        axis=1
    )

    # If 'Duration' is a valid timedelta, convert it to hours, otherwise keep as NaT
    cleaned_temp_df['Duration'] = cleaned_temp_df['Duration'].apply(
        lambda x: round(x.total_seconds() / 3600, 2) if isinstance(x, pd.Timedelta) else "Error, missing value"
    )

    # Merge cleaned_temp_df with new_student_listing to get 'First Name' and 'Last Name'
    merged_df = cleaned_temp_df.merge(new_student_listing[['First Name', 'Last Name']], 
                                      left_on='Name', right_on='First Name', how='left')

    # Populate 'student_id' using get_student_id function
    merged_df['student_id'] = merged_df.apply(
        lambda row: get_student_id(row['First Name'], row['Last Name']), axis=1
    )

    # **Ensure these columns are properly created before dropping**
    merged_df['temp_first_name'] = merged_df['First Name']
    merged_df['temp_last_name'] = merged_df['Last Name']

    # Now you can drop 'First Name' and 'Last Name' if necessary, but retain temp columns
    merged_df = merged_df.drop(columns=['First Name', 'Last Name'])

    # Continue with your existing logic...
    # Reorder columns and clean up missing data
    merged_df['Start DateTime'] = merged_df['Start DateTime'].fillna(default_datetime)
    merged_df['End DateTime'] = merged_df['End DateTime'].fillna(default_datetime)
    merged_df['date'] = merged_df['date'].fillna(default_datetime)
    merged_df = merged_df.fillna("Missing")

    # Ensure columns are in the right order for return
    merged_df = merged_df[['student_id', 'date', 'Start DateTime', 'End DateTime', 
                           'shelves_completed', 'start_call', 'end_call', 'Duration']]

    # Remove rows with 'Missing' values in key columns
    merged_df = merged_df[~((merged_df['date'] == 'Missing') | 
                            (merged_df['Start DateTime'] == 'Missing') | 
                            (merged_df['End DateTime'] == 'Missing') | 
                            (merged_df['student_id'] == 'Missing'))]

    return merged_df

In [141]:
def insert_data_to_db(csv_file_path, excel_file_path):
    print("Starting insert_data_to_db function")  # Debug print to confirm the function is running
    
    floor_name = os.path.basename(excel_file_path).split('_')[1] 
    floor_id = int(floor_name)  

    collection_name = os.path.basename(csv_file_path).split('.')[0] 

    collection = Collections.query.filter_by(collection=collection_name).first()
    if collection is None:
        print(f"Warning: Collection with name {collection_name} not found!")
        collection_id = None
    else:
        collection_id = collection.collection_id

    df = pd.read_csv(csv_file_path)
    inserted_count = 0  # Counter for inserted records

    for _, row in df.iterrows():
        print(f"Processing row: {row}")  # Debug print for each row being processed
        
        if 'temp_first_name' in row and 'temp_last_name' in row:
            student_id = get_student_id(row['temp_first_name'], row['temp_last_name'])
        else:
            print("Error: Missing first or last name in row")
            continue

        if student_id is None:
            print(f"Error: Could not find student ID for {row['temp_first_name']} {row['temp_last_name']}")
            continue

        row.drop(labels=['temp_first_name', 'temp_last_name'], inplace=True)

        existing_record = ShelfReading.query.filter_by(
            date=row['date'],
            start_time=row['start_time'],
            student_id=student_id
        ).first()

        if existing_record is None:
            new_record = ShelfReading(
                date=row['date'],
                start_time=row['start_time'],
                end_time=row['end_time'],
                shelves_completed=row['shelves_completed'],
                start_call=row['start_call'],
                end_call=row['end_call'],
                student_id=student_id,
                floor_id=floor_id,
                collection_id=collection_id
            )
            db.session.add(new_record)
            try:
                db.session.commit()
            except Exception as e:
                print(f"Error committing to the database: {e}")
                db.session.rollback()  # Rollback in case of error
            print(f"Inserted new record for {row['temp_first_name']} {row['temp_last_name']} at floor {floor_id} and collection {collection_name}.")
            inserted_count += 1  # Increment count for successful insertions
        else:
            print(f"Duplicate found for student {student_id} on {row['date']} at {row['start_time']} - Skipping insert.")

    print(f"{inserted_count} records successfully inserted into the database.")  # Ensure this line is reached
    return f"{inserted_count} records successfully inserted into the database."


In [144]:
def process_excel_file(excel_file_path):
    # Read the Excel file
    all_sheets = pd.read_excel(excel_file_path, sheet_name=None)

    # New file path folder based on Excel filename
    folder_name = os.path.splitext(os.path.basename(excel_file_path))[0]
    parent_directory = "processed_files"
    folder_path = os.path.join(parent_directory, folder_name)

    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # Save each sheet as a separate CSV file
    for sheet_name, df in all_sheets.items():
        file_path = os.path.join(folder_path, f"{sheet_name}.csv")
        df.to_csv(file_path, index=False)

    print(f"All sheets saved as CSV files in the folder: {folder_path}")

    # Load student data for merging
    student_listing = pd.read_csv('Oogway-System/Active CMR Student List - Sheet1.csv', header=None)
    student_listing[['Last Name', 'First Name']] = student_listing[0].str.split(',', expand=True)
    student_listing['First Name'] = student_listing['First Name'].str.strip()
    student_listing['Last Name'] = student_listing['Last Name'].str.strip()
    new_student_listing = student_listing.drop(columns=[0])

    # Folder to save edited files
    edited_folder_name = folder_name + "_edited"
    edited_folder_path = os.path.join(parent_directory, edited_folder_name)

    if not os.path.exists(edited_folder_path):
        os.makedirs(edited_folder_path)

    # Process each sheet and save edited CSV files
    edited_files_count = 0  # To count edited files
    for sheet_name, df in all_sheets.items():
        new_df = edit_sheet(df, new_student_listing)
        file_path = os.path.join(edited_folder_path, f"{sheet_name}_edited.csv")
        new_df.to_csv(file_path, index=False)
        print(f"Edited sheet '{sheet_name}' saved to: {file_path}")
        
        # Insert data into the database
        insert_data_to_db(file_path, excel_file_path)  # Pass excel file path to insert data
        edited_files_count += 1

    # Return a summary message
    return f"Processed {len(all_sheets)} sheets, saved {edited_files_count} edited files, and inserted data into the database."


In [143]:
# for sheet_name, df in all_sheets.items():
#     new_df = edit_sheet(df, new_student_listing)
#     print(new_df)
    # file_path = os.path.join(folder_path, f"{sheet_name}_edited.csv")
    # new_df2.to_csv(file_path, index=False)
    
    # print(f"Edited sheet '{sheet_name}' saved to: {file_path}")